In [ ]:
# -*- coding: utf-8 -*-
# EEGNet + protocolo global + fine-tuning progresivo por sujeto (augments + SGDR + tweaks)
# Cambios clave:
# 1) Capacidad ↑: F1=24, fc=128
# 2) Loss ponderada por clase (+20% a Both Fists) con soporte para mixup (WeightedSoftCrossEntropy)
# 3) Cutout focalizado (30–60 ms) solo para clases OF (2,3)
# 4) SGDR con T0=6, Tmult=2
# 5) TTA (n=5, jitter ±25 ms) en evaluación

import os, re, math, random, json, itertools, copy
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import mne

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler

from sklearn.model_selection import GroupKFold, StratifiedKFold, StratifiedShuffleSplit, GroupShuffleSplit
from sklearn.utils import check_random_state
from sklearn.metrics import confusion_matrix, classification_report

import matplotlib.pyplot as plt
from tqdm import tqdm

# =========================
# CONFIGURACIÓN GENERAL
# =========================
PROJ = Path('..').resolve().parent
DATA_RAW = PROJ / 'data' / 'raw'
CACHE_DIR = PROJ / 'data' / 'cache'
FOLDS_DIR = PROJ / 'models' / 'folds' / 'Kfold5.json'
CACHE_DIR.mkdir(parents=True, exist_ok=True)

# Dispositivo y semilla
RANDOM_STATE = 42
torch.manual_seed(RANDOM_STATE); np.random.seed(RANDOM_STATE); random.seed(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Usando dispositivo: {DEVICE}")

# Escenario y ventana
CLASS_SCENARIO = '4c'
WINDOW_MODE = '6s'   # mantienes 6s
FS = 160.0
N_FOLDS = 5

# Entrenamiento global
BATCH_SIZE = 64
EPOCHS_GLOBAL = 100
LR_INIT = 1e-2
SGDR_T0 = 6      # ↓ ciclos cortos al inicio
SGDR_Tmult = 2

# Validación/ES global
GLOBAL_VAL_SPLIT = 0.15
GLOBAL_PATIENCE  = 10
LOG_EVERY        = 5

# Fine-tuning por sujeto (protocolo robusto)
CALIB_CV_FOLDS = 4
FT_EPOCHS = 30
FT_BASE_LR = 5e-5
FT_HEAD_LR = 1e-3
FT_L2SP = 1e-4
FT_PATIENCE = 5
FT_VAL_RATIO = 0.2

# Normalización por época canal-a-canal (z-score)
NORM_EPOCH_ZSCORE = True

# Sujetos excluidos
EXCLUDE_SUBJECTS = {38, 88, 89, 92, 100, 104}

# Runs
MI_RUNS_LR = [4, 8, 12]
MI_RUNS_OF = [6, 10, 14]
BASELINE_RUNS_EO = [1]

# Canales (8 con FCz)
EXPECTED_8 = ['C3','C4','Cz','CP3','CP4','FC3','FC4','FCz']

# =========================
# UTILIDADES DE CANALES
# =========================
def normalize_label(s: str) -> str:
    if s is None: return s
    s = s.strip()
    s = re.sub(r'[^A-Za-z0-9]', '', s)
    s = re.sub(r'([A-Za-z])0([0-9])', r'\1\2', s)
    s = re.sub(r'([A-Za-z])Z$', r'\1z', s)
    s = s.replace('fp', 'Fp').replace('FP', 'Fp')
    s = ''.join(ch.upper() if ch != 'z' else 'z' for ch in s)
    return s

def rename_channels_1010(raw: mne.io.BaseRaw):
    mapping = {}
    for ch in raw.ch_names:
        lab = normalize_label(ch)
        lab = lab[:-1] + 'z' if lab.endswith('Z') else lab
        lab = re.sub(r'([A-Z])Z$', r'\1z', lab)
        mapping[ch] = lab
    mne.rename_channels(raw.info, mapping)

def ensure_channels_order(raw: mne.io.BaseRaw, desired_channels=EXPECTED_8):
    missing = [ch for ch in desired_channels if ch not in raw.ch_names]
    if missing:
        print(f"Warning: faltan canales {missing} en archivo {getattr(raw,'filenames', [''])[0]}")
        return None
    raw.reorder_channels([ch for ch in raw.ch_names if ch in desired_channels] +
                         [ch for ch in raw.ch_names if ch not in desired_channels])
    raw.pick_channels(desired_channels, ordered=True)
    return raw

# =========================
# LECTURA DE EDF y EVENTOS
# =========================
_re_file = re.compile(r'[Ss](\d{3}).*?[Rr](\d{2})')

def parse_subject_run(path: Path):
    m = _re_file.search(str(path))
    if not m: return None, None
    return int(m.group(1)), int(m.group(2))

def run_kind(run_id:int):
    if run_id in MI_RUNS_LR: return 'LR'
    if run_id in MI_RUNS_OF: return 'OF'
    if run_id in BASELINE_RUNS_EO: return 'EO'
    return None

# --- Notch adaptativo: lee CSV si existe (subject, run, snr50_db, snr60_db)
_SNR_TABLE = None
def _load_snr_table():
    global _SNR_TABLE
    if _SNR_TABLE is not None:
        return _SNR_TABLE
    csv_path = PROJ / 'reports' / 'psd_mains' / 'psd_mains_summary.csv'
    if csv_path.exists():
        try:
            df = pd.read_csv(csv_path)
            _SNR_TABLE = df
        except Exception as e:
            print(f"[SNR] No se pudo leer {csv_path}: {e}")
            _SNR_TABLE = None
    return _SNR_TABLE

def _decide_notch(subject, run, th_db=10.0):
    df = _load_snr_table()
    if df is None:  # default
        return 60.0
    row = df[(df['subject']==subject) & (df['run']==run)]
    if row.empty:
        return 60.0
    snr50 = float(row['snr50_db'].iloc[0]); snr60 = float(row['snr60_db'].iloc[0])
    if snr60 >= th_db and snr60 >= snr50: return 60.0
    if snr50 >= th_db and snr50 >  snr60: return 50.0
    return None  # no notch si no sobresale

def read_raw_edf(path: Path):
    raw = mne.io.read_raw_edf(path, preload=True, verbose=False)
    raw.pick(mne.pick_types(raw.info, eeg=True))
    rename_channels_1010(raw)
    try:
        mont = mne.channels.make_standard_montage('standard_1020')
        raw.set_montage(mont, on_missing='ignore')
    except Exception:
        pass
    if abs(raw.info['sfreq'] - FS) > 1e-6:
        raw.resample(FS, npad="auto")
    raw = ensure_channels_order(raw, EXPECTED_8)
    if raw is None:
        return None

    # Notch adaptativo (por SNR); si no hay CSV, por defecto 60 Hz
    sid, rid = parse_subject_run(path)
    notch = _decide_notch(sid, rid)
    if notch is not None:
        raw.notch_filter(freqs=[float(notch)], picks='eeg', method='spectrum_fit', phase='zero')

    return raw

def collect_events_T1T2(raw: mne.io.BaseRaw):
    if raw.annotations is None or len(raw.annotations) == 0:
        return []
    def _norm(s): return str(s).strip().upper().replace(' ', '')
    res = []
    for onset, desc in zip(raw.annotations.onset, raw.annotations.description):
        tag = _norm(desc)
        if tag in ('T1','T2'):
            res.append((float(onset), tag))
    res.sort()
    dedup = []
    last_t1 = last_t2 = -1e9
    for t, tag in res:
        if tag == 'T1':
            if (t - last_t1) >= 0.5: dedup.append((t, tag)); last_t1 = t
        else:
            if (t - last_t2) >= 0.5: dedup.append((t, tag)); last_t2 = t
    return dedup

# =========================
# CONSTRUCCIÓN DE DATASETS
# =========================
def subjects_available():
    subs = []
    for sdir in sorted(DATA_RAW.glob('S*')):
        if not sdir.is_dir(): continue
        try: sid = int(sdir.name[1:])
        except: continue
        if sid in EXCLUDE_SUBJECTS: continue
        any_mi = any((sdir / f"S{sid:03d}R{r:02d}.edf").exists() for r in (MI_RUNS_LR + MI_RUNS_OF))
        if any_mi: subs.append(sid)
    return subs

def extract_trials_from_run(edf_path: Path, scenario: str, window_mode: str):
    subj, run = parse_subject_run(edf_path)
    kind = run_kind(run)
    if kind not in ('LR','OF','EO'):
        return ([], [])

    raw = read_raw_edf(edf_path)
    if raw is None:
        return ([], [])

    data = raw.get_data()
    fs = raw.info['sfreq']
    assert abs(fs - FS) < 1e-6

    out = []

    if kind in ('LR','OF'):
        events = collect_events_T1T2(raw)
        if window_mode == '3s':
            rel_start, rel_end = 0.0, 3.0
        else:
            rel_start, rel_end = -1.0, 5.0

        for onset_sec, tag in events:
            if kind == 'LR':
                if tag == 'T1': label = 'L'
                elif tag == 'T2': label = 'R'
                else: continue
            else:
                if tag == 'T1': label = 'BFISTS'
                elif tag == 'T2': label = 'BFEET'
                else: continue

            if scenario == '2c' and label not in ('L','R'): continue
            if scenario == '3c' and label not in ('L','R','BFISTS'): continue
            if scenario == '4c' and label not in ('L','R','BFISTS','BFEET'): continue

            s = int(round((raw.first_time + onset_sec + rel_start) * fs))
            e = int(round((raw.first_time + onset_sec + rel_end) * fs))
            if s < 0 or e > data.shape[1]:
                continue

            seg = data[:, s:e].T.astype(np.float32)
            if NORM_EPOCH_ZSCORE:
                seg = (seg - seg.mean(axis=0, keepdims=True)) / (seg.std(axis=0, keepdims=True) + 1e-6)

            if label == 'L':       y = 0
            elif label == 'R':     y = 1
            elif label == 'BFISTS':y = 2
            elif label == 'BFEET': y = 3
            else: continue

            out.append((seg, y, subj))

    elif kind == 'EO':
        return ([], raw.ch_names)

    return out, raw.ch_names

def build_dataset_all(subjects, scenario='4c', window_mode='3s'):
    X, y, groups = [], [], []
    ch_template = None

    for s in tqdm(subjects, desc="Construyendo dataset (RAW)"):
        sdir = DATA_RAW / f"S{s:03d}"
        if not sdir.exists(): continue

        trials_L, trials_R, trials_FISTS, trials_FEET = [], [], [], []

        for r in MI_RUNS_LR:
            p = sdir / f"S{s:03d}R{r:02d}.edf"
            if not p.exists(): continue
            outs, chs = extract_trials_from_run(p, scenario, window_mode)
            if ch_template is None and chs: ch_template = chs
            for seg, lab, _ in outs:
                if lab == 0: trials_L.append(seg)
                elif lab == 1: trials_R.append(seg)

        for r in MI_RUNS_OF:
            p = sdir / f"S{s:03d}R{r:02d}.edf"
            if not p.exists(): continue
            outs, chs = extract_trials_from_run(p, scenario, window_mode)
            if ch_template is None and chs: ch_template = chs
            for seg, lab, _ in outs:
                if lab == 2: trials_FISTS.append(seg)
                elif lab == 3: trials_FEET.append(seg)

        need_per_class = 21
        def pick(trials, n, rng):
            if len(trials) < n:
                idx = rng.choice(len(trials), size=n, replace=True)
                return [trials[i] for i in idx]
            rng.shuffle(trials)
            return trials[:n]

        rng = check_random_state(RANDOM_STATE + s)
        if len(trials_L)==0 or len(trials_R)==0 or len(trials_FISTS)==0 or len(trials_FEET)==0:
            continue

        Lp  = pick(trials_L,     need_per_class, rng)
        Rp  = pick(trials_R,     need_per_class, rng)
        FIp = pick(trials_FISTS, need_per_class, rng)
        FEp = pick(trials_FEET,  need_per_class, rng)

        pack = [(Lp, 0), (Rp, 1), (FIp, 2), (FEp, 3)]
        for segs, lab in pack:
            for seg in segs:
                X.append(seg); y.append(lab); groups.append(s)

    X = np.stack(X, axis=0)
    y = np.asarray(y, dtype=np.int64)
    groups = np.asarray(groups, dtype=np.int64)

    n, T, C = X.shape
    n_classes = len(np.unique(y))
    print(f"Dataset construido: N={n} | T={T} | C={C} | clases={n_classes} | sujetos únicos={len(np.unique(groups))}")
    return X, y, groups, ch_template

# =========================
# AUGMENTS (solo train)
# =========================
def do_time_jitter(x, max_ms=50, fs=160.0):
    # x: (B,1,T,C) torch.float
    max_shift = int(round(max_ms/1000.0 * fs))
    if max_shift <= 0: return x
    B,_,T,C = x.shape
    shifts = torch.randint(low=-max_shift, high=max_shift+1, size=(B,), device=x.device)
    out = torch.empty_like(x)
    for i,s in enumerate(shifts):
        if s==0: out[i] = x[i]; continue
        if s>0:
            out[i,:,s:,:] = x[i,:,:T-s,:]
            out[i,:,:s,:] = 0
        else:
            s = -s
            out[i,:,:T-s,:] = x[i,:,s:,:]
            out[i,:,T-s:,:] = 0
    return out

def do_gaussian_noise(x, sigma=0.01):
    if sigma<=0: return x
    return x + sigma*torch.randn_like(x)

def do_temporal_cutout(x, min_ms=100, max_ms=150, fs=160.0):
    B,_,T,C = x.shape
    Lmin = int(round(min_ms/1000.0*fs))
    Lmax = int(round(max_ms/1000.0*fs))
    if Lmin<=0 or Lmax<=0 or Lmin>Lmax: return x
    out = x.clone()
    for i in range(B):
        L = random.randint(Lmin, Lmax)
        if L>=T: continue
        s = random.randint(0, T-L)
        out[i,:,s:s+L,:] = 0
    return out

def do_temporal_cutout_masked(x, y, classes_mask={2,3}, min_ms=30, max_ms=60, fs=160.0):
    # Aplica cutout solo si y pertenece a classes_mask
    B,_,T,C = x.shape
    if B == 0: return x
    Lmin = int(round(min_ms/1000.0*fs))
    Lmax = int(round(max_ms/1000.0*fs))
    if Lmin<=0 or Lmax<=0 or Lmin>Lmax: return x
    out = x.clone()
    y_np = y.detach().cpu().numpy()
    for i in range(B):
        if int(y_np[i]) not in classes_mask: 
            continue
        L = random.randint(Lmin, Lmax)
        if L>=T: continue
        s = random.randint(0, T-L)
        out[i,:,s:s+L,:] = 0
    return out

def mixup_batch(x, y, n_classes, alpha=0.2):
    if alpha<=0:
        y_onehot = torch.nn.functional.one_hot(y, num_classes=n_classes).float()
        return x, y_onehot, 1.0
    lam = np.random.beta(alpha, alpha)
    perm = torch.randperm(x.size(0), device=x.device)
    x_mix = lam*x + (1-lam)*x[perm]
    y_a = torch.nn.functional.one_hot(y, num_classes=n_classes).float()
    y_b = y_a[perm]
    y_mix = lam*y_a + (1-lam)*y_b
    return x_mix, y_mix, lam

class WeightedSoftCrossEntropy(nn.Module):
    """
    Cross-entropy suave (targets en probas, p.ej. mixup) con pesos por clase.
    """
    def __init__(self, class_weights=None, label_smoothing=0.0):
        super().__init__()
        self.register_buffer('w', None if class_weights is None else class_weights.clone().float())
        self.ls = float(label_smoothing)

    def forward(self, logits, target_probs):
        # label smoothing: mezcla target_probs con uniforme
        if self.ls > 0:
            K = logits.size(1)
            target_probs = (1-self.ls)*target_probs + self.ls*(1.0/K)
        logp = torch.log_softmax(logits, dim=1)  # (B,C)
        loss_per_class = -(target_probs * logp)  # (B,C)
        if self.w is not None:
            loss_per_class = loss_per_class * self.w.unsqueeze(0)  # pesar por clase
        loss = loss_per_class.sum(dim=1).mean()
        return loss

# =========================
# EEGNet (Lawhern et al., 2018) adaptado a (B,1,T,C) + ChannelDropout
# =========================
class ChannelDropout(nn.Module):
    def __init__(self, p=0.1):
        super().__init__()
        self.p = p
    def forward(self, x):
        if not self.training or self.p<=0: return x
        B,_,T,C = x.shape
        mask = (torch.rand(B,1,1,C, device=x.device) > self.p).float()
        return x * mask

class EEGNet(nn.Module):
    """
    Entrada: x de forma (B, 1, T, C)
    """
    def __init__(self, n_ch: int, n_classes: int,
                 F1: int = 24, D: int = 2, kernel_t: int = 64, k_sep: int = 16,
                 pool1_t: int = 4, pool2_t: int = 6,
                 drop1_p: float = 0.35, drop2_p: float = 0.6,
                 chdrop_p: float = 0.1):
        super().__init__()
        self.n_ch = n_ch
        self.n_classes = n_classes
        self.F1 = F1
        self.D = D
        self.F2 = F1 * D
        self.kernel_t = kernel_t
        self.k_sep = k_sep
        self.pool1_t = pool1_t
        self.pool2_t = pool2_t

        self.chdrop = ChannelDropout(p=chdrop_p)

        # Bloque 1: temporal
        self.conv_temporal = nn.Conv2d(1, F1, kernel_size=(kernel_t, 1),
                                       padding=(kernel_t // 2, 0), bias=False)
        self.bn1 = nn.BatchNorm2d(F1, momentum=0.99, eps=1e-3)
        self.act = nn.ELU()

        # Bloque 2: depthwise (espacial)
        self.conv_depthwise = nn.Conv2d(F1, self.F2, kernel_size=(1, n_ch),
                                        groups=F1, bias=False)
        self.bn2 = nn.BatchNorm2d(self.F2, momentum=0.99, eps=1e-3)
        self.pool1 = nn.AvgPool2d(kernel_size=(pool1_t, 1), stride=(pool1_t, 1))
        self.drop1 = nn.Dropout(drop1_p)

        # Bloque 3: separable temporal
        self.conv_sep_depth = nn.Conv2d(self.F2, self.F2, kernel_size=(k_sep, 1),
                                        groups=self.F2, padding=(k_sep // 2, 0), bias=False)
        self.conv_sep_point = nn.Conv2d(self.F2, self.F2, kernel_size=(1, 1), bias=False)
        self.bn3 = nn.BatchNorm2d(self.F2, momentum=0.99, eps=1e-3)
        self.pool2 = nn.AvgPool2d(kernel_size=(pool2_t, 1), stride=(pool2_t, 1))
        self.drop2 = nn.Dropout(drop2_p)

        self.flatten = nn.Flatten()

        # Cabeza dinámica
        self.fc = None
        self.out = None
        self._T_in = None

    def _build_head(self, T_in: int, device: torch.device):
        T1 = T_in // self.pool1_t
        T2 = T1 // self.pool2_t
        feat_dim = self.F2 * T2 * 1
        self.fc  = nn.Linear(feat_dim, 128, bias=True).to(device)
        self.out = nn.Linear(128, self.n_classes, bias=True).to(device)
        self._T_in = T_in

    def ensure_head(self, T_in: int, device: torch.device):
        if (self.fc is None) or (self.out is None) or (self._T_in != T_in):
            self._build_head(T_in, device)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, _, T, C = x.shape
        self.ensure_head(T, x.device)

        x = self.chdrop(x)  # ChannelDropout

        z = self.conv_temporal(x)
        z = self.bn1(z); z = self.act(z)

        z = self.conv_depthwise(z)   # (B, F2, T, 1)
        z = self.bn2(z); z = self.act(z)
        z = self.pool1(z)
        z = self.drop1(z)

        z = self.conv_sep_depth(z)
        z = self.conv_sep_point(z)
        z = self.bn3(z); z = self.act(z)
        z = self.pool2(z)
        z = self.drop2(z)

        z = self.flatten(z)
        z = self.fc(z); z = self.act(z)
        z = self.out(z)
        return z

# =========================
# TORCH DATASET
# =========================
class EEGTrials(Dataset):
    def __init__(self, X, y, groups):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.int64)
        self.g = groups.astype(np.int64)
    def __len__(self): return self.X.shape[0]
    def __getitem__(self, idx):
        x = self.X[idx]
        x = np.expand_dims(x, 0)                 # (1, T, C)
        return torch.from_numpy(x), torch.tensor(self.y[idx]), torch.tensor(self.g[idx])

CLASS_NAMES_4C = ['Left', 'Right', 'Both Fists', 'Both Feet']

# =========================
# UTIL: MAX-NORM
# =========================
@torch.no_grad()
def apply_max_norm(model, max_value=2.0, p=2.0):
    layers = []
    if hasattr(model, 'conv_depthwise'): layers.append(model.conv_depthwise)
    if hasattr(model, 'conv_sep_point'): layers.append(model.conv_sep_point)
    if hasattr(model, 'fc'):             layers.append(model.fc)
    if hasattr(model, 'out'):            layers.append(model.out)
    for layer in layers:
        if hasattr(layer, 'weight') and layer.weight is not None:
            w = layer.weight.data
            norms = w.view(w.size(0), -1).norm(p=p, dim=1, keepdim=True)
            desired = torch.clamp(norms, max=max_value)
            w.view(w.size(0), -1).mul_(desired / (1e-8 + norms))

# =========================
# ENTRENAMIENTO / EVALUACIÓN
# =========================
def build_weighted_sampler(y, groups):
    # pesos inversos por clase y por sujeto → promueve balance en cada batch
    y = np.asarray(y); groups = np.asarray(groups)
    class_counts = np.bincount(y, minlength=len(np.unique(y))).astype(float)
    class_w = 1.0 / class_counts[y]
    subj_vals, subj_counts = np.unique(groups, return_counts=True)
    subj_map = {s:c for s,c in zip(subj_vals, subj_counts)}
    subj_w = np.array([1.0/subj_map[g] for g in groups], dtype=float)
    w = class_w * subj_w
    w = w / w.mean()
    w_t = torch.from_numpy(w).float()
    sampler = WeightedRandomSampler(weights=w_t, num_samples=len(w_t), replacement=True)
    return sampler

def make_class_weight_tensor(y_indices, n_classes, boost_bfists=1.20):
    # pesos inversos por clase, normalizados; clase 2 (*Both Fists*) con boost
    counts = np.bincount(y_indices, minlength=n_classes).astype(np.float32)
    counts[counts == 0] = 1.0
    w = counts.sum() / counts
    w = w / w.mean()
    w[2] *= boost_bfists
    w = w / w.mean()
    return torch.tensor(w, dtype=torch.float32, device=DEVICE)

def train_epoch(model, loader, opt, criterion_soft, n_classes,
                do_aug=True, fs=160.0, maxnorm=None):
    model.train()
    for xb, yb, _ in loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)

        # ---- AUGMENTS (orden: jitter → noise → cutout focalizado → mixup) ----
        if do_aug:
            xb = do_time_jitter(xb, max_ms=50, fs=fs)
            xb = do_gaussian_noise(xb, sigma=0.01)
            xb = do_temporal_cutout_masked(xb, yb, classes_mask={2,3}, min_ms=30, max_ms=60, fs=fs)
            xb, yt, _ = mixup_batch(xb, yb, n_classes=n_classes, alpha=0.2)
            logits = model(xb)
            loss = criterion_soft(logits, yt)
        else:
            logits = model(xb)
            # yb como índices → convertir a one-hot para criterio suave
            yt = torch.nn.functional.one_hot(yb, num_classes=n_classes).float()
            loss = criterion_soft(logits, yt)

        opt.zero_grad(set_to_none=True)
        loss.backward()
        opt.step()
        if maxnorm is not None:
            apply_max_norm(model, max_value=maxnorm, p=2.0)

@torch.no_grad()
def _predict_tta(model, xb, n=5, fs=160.0):
    outs = []
    for _ in range(n):
        xj = do_time_jitter(xb, max_ms=25, fs=fs)
        outs.append(model(xj))
    return torch.stack(outs, dim=0).mean(dim=0)

@torch.no_grad()
def evaluate_with_preds(model, loader, use_tta=True, tta_n=5):
    model.eval()
    y_true, y_pred = [], []
    for xb, yb, _ in loader:
        xb = xb.to(DEVICE)
        if use_tta:
            logits = _predict_tta(model, xb, n=tta_n, fs=FS)
        else:
            logits = model(xb)
        pred = logits.argmax(dim=1).cpu().numpy().tolist()
        y_pred.extend(pred)
        y_true.extend(yb.numpy().tolist())
    y_true = np.asarray(y_true, dtype=int)
    y_pred = np.asarray(y_pred, dtype=int)
    acc = (y_true == y_pred).mean()
    return y_true, y_pred, float(acc)

def plot_confusion(y_true, y_pred, classes, title, fname):
    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(classes))))
    with np.errstate(invalid='ignore'):
        cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
    cm_norm = np.nan_to_num(cm_norm)

    plt.figure(figsize=(6, 5))
    plt.imshow(cm_norm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)
    plt.colorbar(fraction=0.046, pad=0.04)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, ha='right')
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm_norm.max() / 2.
    for i, j in itertools.product(range(cm_norm.shape[0]), range(cm_norm.shape[1])):
        plt.text(j, i, format(cm_norm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm_norm[i, j] > thresh else "black")

    plt.ylabel('True label'); plt.xlabel('Predicted label')
    plt.tight_layout(); plt.savefig(fname, dpi=150, bbox_inches='tight'); plt.close()

def plot_training_curves(history, fname):
    plt.figure(figsize=(6,4))
    plt.plot(history['train_acc'], label='train_acc')
    plt.plot(history['val_acc'], label='val_acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy')
    plt.title('Training curve'); plt.legend()
    plt.tight_layout(); plt.savefig(fname, dpi=150); plt.close()

def print_report(y_true, y_pred, classes):
    print(classification_report(y_true, y_pred, target_names=classes, digits=4))

# =========================
# FINE-TUNING PROGRESIVO por sujeto (con validación interna)
# =========================
def _param_groups(model, mode):
    if mode == 'out':
        train = list(model.out.parameters())
    elif mode == 'head':
        train = list(model.fc.parameters()) + list(model.out.parameters())
    elif mode == 'spatial+head':
        train = (list(model.conv_depthwise.parameters()) +
                 list(model.bn2.parameters()) +
                 list(model.conv_sep_depth.parameters()) +
                 list(model.conv_sep_point.parameters()) +
                 list(model.bn3.parameters()) +
                 list(model.fc.parameters()) +
                 list(model.out.parameters()))
    else:
        raise ValueError(mode)
    return train

def _freeze_for_mode(model, mode):
    for p in model.parameters(): p.requires_grad = False
    if mode == 'out':
        for p in model.out.parameters(): p.requires_grad = True
    elif mode == 'head':
        for p in model.fc.parameters():  p.requires_grad = True
        for p in model.out.parameters(): p.requires_grad = True
    elif mode == 'spatial+head':
        for p in model.conv_depthwise.parameters(): p.requires_grad = True
        for p in model.bn2.parameters():           p.requires_grad = True
        for p in model.conv_sep_depth.parameters():p.requires_grad = True
        for p in model.conv_sep_point.parameters():p.requires_grad = True
        for p in model.bn3.parameters():           p.requires_grad = True
        for p in model.fc.parameters():            p.requires_grad = True
        for p in model.out.parameters():           p.requires_grad = True

def _class_weights(y_np, n_classes):
    counts = np.bincount(y_np, minlength=n_classes).astype(np.float32)
    counts[counts == 0] = 1.0
    weights = counts.sum() / counts
    weights = weights / weights.mean()
    return torch.tensor(weights, dtype=torch.float32, device=DEVICE)

def _train_one_mode(model, X_cal, y_cal, n_classes, mode,
                    epochs=FT_EPOCHS, batch_size=32,
                    head_lr=FT_HEAD_LR, base_lr=FT_BASE_LR,
                    l2sp_lambda=FT_L2SP, patience=FT_PATIENCE, val_ratio=FT_VAL_RATIO):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=val_ratio, random_state=RANDOM_STATE)
    (tr_idx, va_idx), = sss.split(X_cal, y_cal)
    Xtr, ytr = X_cal[tr_idx], y_cal[tr_idx]
    Xva, yva = X_cal[va_idx], y_cal[va_idx]

    ds_tr = torch.utils.data.TensorDataset(
        torch.from_numpy(Xtr).float().unsqueeze(1),
        torch.from_numpy(ytr).long()
    )
    ds_va = torch.utils.data.TensorDataset(
        torch.from_numpy(Xva).float().unsqueeze(1),
        torch.from_numpy(yva).long()
    )
    dl_tr = DataLoader(ds_tr, batch_size=batch_size, shuffle=True, drop_last=False)
    dl_va = DataLoader(ds_va, batch_size=batch_size, shuffle=False, drop_last=False)

    _freeze_for_mode(model, mode)

    if mode == 'spatial+head':
        base_params = (list(model.conv_depthwise.parameters()) +
                       list(model.bn2.parameters()) +
                       list(model.conv_sep_depth.parameters()) +
                       list(model.conv_sep_point.parameters()) +
                       list(model.bn3.parameters()))
        head_params = list(model.fc.parameters()) + list(model.out.parameters())
        train_params = base_params + head_params
        opt = optim.Adam([
            {"params": base_params, "lr": base_lr},
            {"params": head_params, "lr": head_lr},
        ])
    else:
        train_params = _param_groups(model, mode)
        opt = optim.Adam(train_params, lr=head_lr)

    ref = [p.detach().clone().to(p.device) for p in train_params]
    class_w = _class_weights(ytr, n_classes)
    crit = nn.CrossEntropyLoss(weight=class_w)

    best_state = copy.deepcopy(model.state_dict())
    best_val = float('inf'); bad = 0

    for _ in range(epochs):
        model.train()
        for xb, yb in dl_tr:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad(set_to_none=True)
            logits = model(xb)
            loss = crit(logits, yb)
            reg = 0.0
            for p_cur, p_ref in zip(train_params, ref):
                reg = reg + torch.sum((p_cur - p_ref)**2)
            loss = loss + l2sp_lambda * reg
            loss.backward(); opt.step()
            apply_max_norm(model, max_value=2.0, p=2.0)

        model.eval()
        with torch.no_grad():
            val_loss = 0.0; nval = 0
            for xb, yb in dl_va:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                logits = model(xb)
                loss = crit(logits, yb)
                val_loss += loss.item() * xb.size(0); nval += xb.size(0)
            val_loss /= max(1, nval)

        if val_loss + 1e-7 < best_val:
            best_val = val_loss; bad = 0
            best_state = copy.deepcopy(model.state_dict())
        else:
            bad += 1
            if bad >= patience: break

    model.load_state_dict(best_state)
    return model

@torch.no_grad()
def predict_numpy(model, X_np, device):
    model.eval()
    xb = torch.from_numpy(X_np).float().unsqueeze(1).to(device)
    logits = model(xb)
    return logits.argmax(dim=1).cpu().numpy()

def subject_cv_finetune_predict_progressive(model_global, Xs, ys, device,
                                            n_splits=CALIB_CV_FOLDS, n_classes=4):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    y_true_full = np.empty_like(ys); y_pred_full = np.empty_like(ys)

    for tr_idx, te_idx in skf.split(Xs, ys):
        Xcal, ycal = Xs[tr_idx], ys[tr_idx]
        Xho,  yho  = Xs[te_idx], ys[te_idx]

        m_out = copy.deepcopy(model_global)
        _train_one_mode(m_out, Xcal, ycal, n_classes, mode='out',
                        epochs=FT_EPOCHS, head_lr=FT_HEAD_LR, l2sp_lambda=FT_L2SP,
                        patience=FT_PATIENCE, val_ratio=FT_VAL_RATIO)
        yhat_out = predict_numpy(m_out, Xho, device); acc_out = (yhat_out == yho).mean()

        m_head = copy.deepcopy(model_global)
        _train_one_mode(m_head, Xcal, ycal, n_classes, mode='head',
                        epochs=FT_EPOCHS, head_lr=FT_HEAD_LR, l2sp_lambda=FT_L2SP,
                        patience=FT_PATIENCE, val_ratio=FT_VAL_RATIO)
        yhat_head = predict_numpy(m_head, Xho, device); acc_head = (yhat_head == yho).mean()

        m_sp = copy.deepcopy(model_global)
        _train_one_mode(m_sp, Xcal, ycal, n_classes, mode='spatial+head',
                        epochs=FT_EPOCHS, head_lr=FT_HEAD_LR, base_lr=FT_BASE_LR,
                        l2sp_lambda=FT_L2SP, patience=FT_PATIENCE, val_ratio=FT_VAL_RATIO)
        yhat_sp = predict_numpy(m_sp, Xho, device); acc_sp = (yhat_sp == yho).mean()

        best_idx = np.argmax([acc_out, acc_head, acc_sp])
        yhat_best = [yhat_out, yhat_head, yhat_sp][best_idx]

        y_true_full[te_idx] = yho; y_pred_full[te_idx] = yhat_best

    return y_true_full, y_pred_full

# =========================
# FOLDS JSON helpers
# =========================
def save_group_folds_json_with_indices(subject_ids_str, groups_array, n_splits, out_json_path,
                                       created_by="dose_experiment", description=None):
    out_json_path = Path(out_json_path)
    unique_subjects_int = sorted(np.unique(groups_array).tolist())
    subject_ids = [f"S{sid:03d}" for sid in unique_subjects_int]

    if len(subject_ids) < n_splits:
        raise ValueError(f"n_splits={n_splits} mayor que número de sujetos={len(subject_ids)}")

    groups = np.arange(len(subject_ids))
    gkf = GroupKFold(n_splits=n_splits)

    folds = []
    fold_i = 0
    for train_idx_grp, test_idx_grp in gkf.split(groups, groups, groups):
        fold_i += 1
        train_sids = [subject_ids[int(i)] for i in train_idx_grp]
        test_sids  = [subject_ids[int(i)] for i in test_idx_grp]

        train_sids_int = [int(s[1:]) for s in train_sids]
        test_sids_int  = [int(s[1:]) for s in test_sids]

        tr_idx = np.where(np.isin(groups_array, train_sids_int))[0].tolist()
        te_idx = np.where(np.isin(groups_array, test_sids_int))[0].tolist()

        folds.append({
            "fold": int(fold_i),
            "train": train_sids,
            "test": test_sids,
            "tr_idx": tr_idx,
            "te_idx": te_idx
        })

    payload = {
        "created_at": datetime.now().isoformat(),
        "created_by": created_by,
        "description": description if description is not None else "",
        "n_splits": int(n_splits),
        "n_subjects": len(subject_ids),
        "subject_ids": subject_ids,
        "folds": folds
    }

    out_json_path.parent.mkdir(parents=True, exist_ok=True)
    with open(out_json_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2, ensure_ascii=False)

    print(f"Folds JSON con índices guardado → {out_json_path}")
    return out_json_path

def load_group_folds_json(path_json, expected_subject_ids=None, strict_check=True):
    path_json = Path(path_json)
    if not path_json.exists():
        raise FileNotFoundError(f"No existe {path_json}")
    with open(path_json, "r", encoding="utf-8") as f:
        payload = json.load(f)

    subj_json = payload.get("subject_ids", [])
    if expected_subject_ids is not None:
        expected = sorted(list(expected_subject_ids))
        if subj_json != expected:
            msg = ("Los subject_ids del JSON no coinciden con expected_subject_ids.\n"
                   f"JSON has {len(subj_json)} subjects, expected {len(expected)}.\n"
                   f"First 10 JSON: {subj_json[:10]}\nFirst 10 expected: {expected[:10]}")
            if strict_check:
                raise ValueError(msg)
            else:
                print("WARNING: " + msg)
    return payload

# =========================
# EXPERIMENTO
# =========================
def run_experiment(save_folds_json=True, folds_json_path=FOLDS_DIR, folds_json_description="GroupKFold folds for comparison"):
    mne.set_log_level('WARNING')

    subs = subjects_available()
    print(f"Sujetos elegibles: {len(subs)} → {subs[:10]}{'...' if len(subs)>10 else ''}")

    X, y, groups, chs = build_dataset_all(subs, scenario=CLASS_SCENARIO, window_mode=WINDOW_MODE)
    N, T, C = X.shape
    n_classes = len(np.unique(y))
    print(f"Listo para entrenar: N={N} | T={T} | C={C} | clases={n_classes} | sujetos={len(np.unique(groups))}")

    ds = EEGTrials(X, y, groups)

    # preparar JSON folds
    if folds_json_path is None:
        folds_json_path = Path("folds") / f"group_folds_{N_FOLDS}splits.json"
    else:
        folds_json_path = Path(folds_json_path)
    folds_json_path.parent.mkdir(parents=True, exist_ok=True)

    unique_subs = sorted(np.unique(groups).tolist())
    subject_ids_str = [f"S{s:03d}" for s in unique_subs]

    if not folds_json_path.exists():
        if not save_folds_json:
            raise FileNotFoundError(f"Folds JSON no encontrado en {folds_json_path} y save_folds_json=False.")
        save_group_folds_json_with_indices(subject_ids_str, groups, n_splits=N_FOLDS,
                                           out_json_path=folds_json_path,
                                           created_by="Joel_Clasificador",
                                           description=folds_json_description)

    payload = load_group_folds_json(folds_json_path, expected_subject_ids=subject_ids_str, strict_check=False)
    folds = payload["folds"]

    # bucle por folds
    global_folds = []
    ft_prog_folds = []
    all_true = []
    all_pred = []

    for f in folds:
        fold = f["fold"]
        tr_idx = np.asarray(f.get("tr_idx", []), dtype=int)
        te_idx = np.asarray(f.get("te_idx", []), dtype=int)

        if tr_idx.size == 0 or te_idx.size == 0:
            print(f"Advertencia: fold {fold} sin índices tr/te válidos. Saltando.")
            continue

        # ===== Split de validación por sujetos =====
        gss = GroupShuffleSplit(n_splits=1, test_size=GLOBAL_VAL_SPLIT, random_state=RANDOM_STATE)
        tr_subj_idx, va_subj_idx = next(gss.split(tr_idx, groups[tr_idx], groups[tr_idx]))
        tr_sub_idx = tr_idx[tr_subj_idx]
        va_idx     = tr_idx[va_subj_idx]

        # Sampler balanceado para train
        sampler = build_weighted_sampler(y[tr_sub_idx], groups[tr_sub_idx])

        tr_loader = DataLoader(Subset(ds, tr_sub_idx), batch_size=BATCH_SIZE, sampler=sampler, drop_last=False)
        va_loader = DataLoader(Subset(ds, va_idx),     batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
        te_loader = DataLoader(Subset(ds, te_idx),     batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

        # ===== EEGNet =====
        model = EEGNet(n_ch=C, n_classes=n_classes,
                       F1=24, D=2, kernel_t=64, k_sep=16,
                       pool1_t=4, pool2_t=6, drop1_p=0.35, drop2_p=0.6,
                       chdrop_p=0.1).to(DEVICE)

        # Opt y scheduler SGDR
        opt = optim.Adam(model.parameters(), lr=LR_INIT)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(opt, T_0=SGDR_T0, T_mult=SGDR_Tmult)

        # Métrica rápida
        def _acc(loader):
            return evaluate_with_preds(model, loader, use_tta=True, tta_n=5)[2]

        # Historia para curvas
        history = {'train_acc': [], 'val_acc': []}

        # ---- Criterio suave con ponderación por clase (+20% BFISTS) ----
        class_weights = make_class_weight_tensor(y[tr_sub_idx], n_classes, boost_bfists=1.20)
        criterion_soft = WeightedSoftCrossEntropy(class_weights, label_smoothing=0.05)

        # ===== Entrenamiento global =====
        print(f"\n[Fold {fold}/{N_FOLDS}] Entrenando modelo global..."
              f" (n_train={len(tr_sub_idx)} | n_val={len(va_idx)} | n_test={len(te_idx)})")

        best_state = copy.deepcopy(model.state_dict())
        best_val = -1.0
        bad = 0

        for epoch in range(1, EPOCHS_GLOBAL + 1):
            train_epoch(model, tr_loader, opt, criterion_soft, n_classes=n_classes,
                        do_aug=True, fs=FS, maxnorm=2.0)
            scheduler.step(epoch-1 + 1e-8)  # tick suave

            # eval
            tr_acc = _acc(tr_loader)
            va_acc = _acc(va_loader)
            history['train_acc'].append(tr_acc)
            history['val_acc'].append(va_acc)

            if (epoch % LOG_EVERY == 0) or epoch in (1, 10, 20, 50, 100):
                cur_lr = opt.param_groups[0]['lr']
                print(f"  Época {epoch:3d} | train_acc={tr_acc:.4f} | val_acc={va_acc:.4f} | LR={cur_lr:.5f}")

            if va_acc > best_val + 1e-4:
                best_val = va_acc
                best_state = copy.deepcopy(model.state_dict())
                bad = 0
            else:
                bad += 1
                if bad >= GLOBAL_PATIENCE:
                    print(f"  Early stopping en época {epoch} (mejor val_acc={best_val:.4f})")
                    break

        # guardar curva de entrenamiento
        curve_path = f"training_curve_fold{fold}.png"
        plot_training_curves(history, curve_path)
        print(f"↳ Curva de entrenamiento guardada: {curve_path}")

        # cargar mejor estado antes de evaluar en test
        model.load_state_dict(best_state)

        # ===== Evaluación global (inter-sujeto puro) =====
        y_true, y_pred, acc_global = evaluate_with_preds(model, te_loader, use_tta=True, tta_n=5)
        global_folds.append(acc_global)
        all_true.append(y_true); all_pred.append(y_pred)

        print(f"[Fold {fold}/{N_FOLDS}] Global acc={acc_global:.4f}")
        print_report(y_true, y_pred, CLASS_NAMES_4C)

        # ---------- Fine-tuning PROGRESIVO por sujeto con 4-fold CV ----------
        X_te, y_te, g_te = X[te_idx], y[te_idx], groups[te_idx]

        y_true_ft_all, y_pred_ft_all = [], []
        used_subjects = 0
        for sid in np.unique(g_te):
            idx = np.where(g_te == sid)[0]
            Xs, ys = X_te[idx], y_te[idx]

            if len(ys) < CALIB_CV_FOLDS or len(np.unique(ys)) < 2:
                continue

            y_true_subj, y_pred_subj = subject_cv_finetune_predict_progressive(
                model, Xs, ys, DEVICE, n_splits=CALIB_CV_FOLDS, n_classes=n_classes
            )
            y_true_ft_all.append(y_true_subj)
            y_pred_ft_all.append(y_pred_subj)
            used_subjects += 1

        if len(y_true_ft_all) > 0:
            y_true_ft_all = np.concatenate(y_true_ft_all)
            y_pred_ft_all = np.concatenate(y_pred_ft_all)
            acc_ft = (y_true_ft_all == y_pred_ft_all).mean()
            print(f"  Fine-tuning PROGRESIVO (por sujeto, {CALIB_CV_FOLDS}-fold CV) acc={acc_ft:.4f} | sujetos={used_subjects}")
            print(f"  Δ(FT-Global) = {acc_ft - acc_global:+.4f}")
        else:
            acc_ft = np.nan
            print("  Fine-tuning PROGRESIVO no ejecutado (sujeto(s) con muestras insuficientes).")

        ft_prog_folds.append(acc_ft)

    # ---------- resultados finales ----------
    if len(all_true) > 0:
        all_true = np.concatenate(all_true)
        all_pred = np.concatenate(all_pred)
    else:
        all_true = np.array([], dtype=int)
        all_pred = np.array([], dtype=int)

    print("\n" + "="*60)
    print("RESULTADOS FINALES")
    print("="*60)
    print("Global folds:", [f"{a:.4f}" for a in global_folds])
    if len(global_folds) > 0:
        print(f"Global mean: {np.mean(global_folds):.4f}")

    print("Fine-tune PROGRESIVO folds:", [("nan" if (a is None or np.isnan(a)) else f"{a:.4f}") for a in ft_prog_folds])
    if len(ft_prog_folds) > 0:
        print(f"Fine-tune PROGRESIVO mean: {np.nanmean(ft_prog_folds):.4f}")
        print(f"Δ(FT-Global) mean: {np.nanmean(ft_prog_folds) - np.mean(global_folds):+.4f}")

    if all_true.size > 0:
        plot_confusion(all_true, all_pred, CLASS_NAMES_4C,
                       title="Confusion Matrix - Global Model (All Folds)",
                       fname="confusion_global_allfolds.png")
        print("\n↳ Matriz de confusión guardada: confusion_global_allfolds.png")

    return {
        "global_folds": global_folds,
        "ft_prog_folds": ft_prog_folds,
        "all_true": all_true,
        "all_pred": all_pred,
        "folds_json_path": str(folds_json_path)
    }

# ---------- MAIN ----------
if __name__ == "__main__":
    print("🧠 INICIANDO EXPERIMENTO CON EEGNet + FINE-TUNING PROGRESIVO (augments + SGDR + tweaks)")
    print(f"🔧 Configuración: {CLASS_SCENARIO}, {len(EXPECTED_8)} canales, {WINDOW_MODE}")
    print(f"⚙️  FT: epochs={FT_EPOCHS}, base_lr={FT_BASE_LR}, head_lr={FT_HEAD_LR}, L2SP={FT_L2SP}, patience={FT_PATIENCE}, CV={CALIB_CV_FOLDS}")
    run_experiment()


🚀 Usando dispositivo: cuda
🧠 INICIANDO EXPERIMENTO CON EEGNet + FINE-TUNING PROGRESIVO (augments + SGDR + tweaks)
🔧 Configuración: 4c, 8 canales, 6s
⚙️  FT: epochs=30, base_lr=5e-05, head_lr=0.001, L2SP=0.0001, patience=5, CV=4
Sujetos elegibles: 103 → [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]...


Construyendo dataset (RAW): 100%|██████████| 103/103 [00:41<00:00,  2.50it/s]


Dataset construido: N=8652 | T=960 | C=8 | clases=4 | sujetos únicos=103
Listo para entrenar: N=8652 | T=960 | C=8 | clases=4 | sujetos=103

[Fold 1/5] Entrenando modelo global... (n_train=5796 | n_val=1092 | n_test=1764)
  Época   1 | train_acc=0.4605 | val_acc=0.4304 | LR=0.01000
  Época   5 | train_acc=0.5138 | val_acc=0.4643 | LR=0.00250
  Época  10 | train_acc=0.5226 | val_acc=0.4744 | LR=0.00854
  Época  15 | train_acc=0.5514 | val_acc=0.4872 | LR=0.00250
  Época  20 | train_acc=0.5371 | val_acc=0.4698 | LR=0.00996
  Época  25 | train_acc=0.5229 | val_acc=0.4799 | LR=0.00854
  Early stopping en época 26 (mejor val_acc=0.4881)
↳ Curva de entrenamiento guardada: training_curve_fold1.png
[Fold 1/5] Global acc=0.4575
              precision    recall  f1-score   support

        Left     0.5516    0.4603    0.5019       441
       Right     0.5642    0.4785    0.5178       441
  Both Fists     0.3540    0.5057    0.4164       441
   Both Feet     0.4337    0.3855    0.4082       441
